In [2]:
#  IMPORTS
import sys
import datetime

# a utility 
from fasp.runner import FASPRunner

# The implementations we're using
from fasp.search import DataConnectClient
from fasp.loc import DRSClient
from fasp.workflow import DNAStackWESClient


# Step 1 - Discovery
# query for relevant DRS objects
searchClient = DataConnectClient('https://ga4gh-search-adapter-presto-public.prod.dnastack.com/')
query = '''select submitter_id, read_drs_id drsid 
from thousand_genomes.onek_genomes.ssd_drs 
where population = 'ACB' limit 3'''
query_job = searchClient.runQuery(query)
query_job

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________


[['HG01988_demo', 'dg.4503/7c7232a4-21d3-4d25-9846-bc0fe0547cee'],
 ['HG01986_demo', 'dg.4503/0e552602-5cec-477e-baa8-f1b1ed6a5bcb'],
 ['HG02317_demo', 'dg.4503/732731e4-4ef4-4874-82bd-d6a038ddc81f']]

In [ ]:
# Step 2 - DRS - set up a DRS Client
# CRDC
drsClient = sdlDRSClient('~/.keys/prj_11218_D17199.ngc')

# Step 3 - set up a class that run a compute for us
wesClient = DNAStackWESClient('~/.keys/dnastack_wes_credentials.json')

# A log is helpful to keep track of the computes we've submitted
faspRunner = FASPRunner()

# repeat steps 2 and 3 for each row of the query
for row in query_job:

    print("subject={}, drsID={}".format(row[0], row[1]))

    # Step 2 - Use DRS to get the URL
    #objInfo = drsClient.getObject(row[1])
    # for testing
    acc = 'SRR5368359.sra'
    objInfo = drsClient.getObject(acc)
    fileSize = objInfo['size']
    print(fileSize)
    # we've predetermined we want to use the gs copy in this case
    #url = drsClient.getAccessURL(row[1], 'gs')
    res = drsClient.getAccessURL(acc,'gs.us')
    url = res['url']
    print(url)
    # Step 3 - Run a pipeline on the file at the drs url
    outfile = "{}.txt".format(row[0])
    pipeline_id = wesClient.runWorkflow(url, outfile)
    print('submitted:{}'.format(pipeline_id))

    via = 'WES'
    note = 'WES MD5 on NCBI SDL'

    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    faspRunner.logRun(time, via, note,  pipeline_id, outfile, str(fileSize),
        searchClient, drsClient, wesClient)


